In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from result_saver import SaverProvider
provider = SaverProvider()

In [3]:
import numpy as np
from Scratch import metadata_loader

md = metadata_loader(_extract=True, _drop_inutile=True)
md = md[md["job_status"] == "JobStatus.DONE"]
md = md[md["notebook_name"] == "bigger_rep_codes"]
max_distance = int(max(md.distance))
max_distance = 10
md = md[md["distance"] == max_distance]
md = md.sort_values(by='backend_name', ascending=False)

md = md[:2]

md

,creation_date,notebook_name,backend_name,job_id,tags,shots,tags_xp,sampled_state,num_qubits,job_status,extra,optimization_level,code,distance,rounds,logical,layout,descr
51,2023-10-29 14:43:41.752908+01:00,bigger_rep_codes,ibm_sherbrooke,cmz633cvayrg008sp2a0,[],10000.0,NaN,NaN,NaN,JobStatus.DONE,NaN,NaN,RepetitionCodeCircuit,10.0,10,0,_is_hex=True,Run bigger Repetition codes
242,2023-10-30 09:43:20.984078+01:00,bigger_rep_codes,ibm_sherbrooke,cmzpsa5vayrg008cpk30,[],10000.0,NaN,NaN,NaN,JobStatus.DONE,NaN,NaN,RepetitionCodeCircuit,10.0,10,0,_is_hex=True,Run bigger Repetition codes v4: new distances ...


In [4]:
memories = {}
for job_id, logical in zip(md.job_id, md.logical):
    mmr_name = f"mmr_log_{logical}"
    memories[mmr_name] = provider.retrieve_job(job_id).result().get_memory()

memory = memories['mmr_log_0']

# UF

In [5]:
import pickle
from soft_info import get_repcode_layout, get_KDEs, get_counts
from qiskit_qec.circuits import RepetitionCodeCircuit
from qiskit_qec.decoders import UnionFindDecoder

layout = get_repcode_layout(distance=max_distance, backend=provider.get_backend("ibm_sherbrooke"), _is_hex=True)

kde_dict, scaler_dict = get_KDEs(provider, 'ibm_sherbrooke', layout, bandwidths=0.2, plot=False)
print(kde_dict)

code = RepetitionCodeCircuit(max_distance, max_distance)
#decoder = UnionFindDecoder(code)

with open(f'dist_{max_distance}_decoder.pkl', 'rb') as f:
    decoder = pickle.load(f)

{114: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 116: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 118: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 120: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 122: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 104: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 102: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 100: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 98: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 113: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 115: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 117: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 119: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 121: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 111: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.

In [19]:
from soft_info import soft_reweight, get_counts, rx_draw_2D
from tqdm import tqdm

VERBOSE = False

with open(f"dist_{max_distance}_decoder.pkl", 'rb') as f: # Reload the decoder each time to reset the graph
        decoder = pickle.load(f)


logical_counts = {'0':0, '1':0}
for shot in tqdm(range(len(memory))[:]):
    
    # with open(f"dist_{max_distance}_decoder.pkl", 'rb') as f: # Reload the decoder each time to reset the graph
    #     decoder = pickle.load(f)

    IQ_data = memory[shot]

    counts = get_counts([IQ_data], kde_dict, scaler_dict, layout, max_distance, verbose=False)
    count_key = next(iter(counts.keys()))
    
    

    #reweighted_graph = soft_reweight(decoder, IQ_data, kde_dict, scaler_dict, layout)
    #decoder.decoding_graph.graph = reweighted_graph

    

    logical, flipped_qubit_dict = decoder.process(count_key, _return_err_str = True)
    
    logical_counts[f"{logical[0]}"] += 1

    if logical[0] == 0:
        continue
    
    print(logical_counts)

    if VERBOSE:
        print("Decoding graph after reweighting:")
        rx_draw_2D(decoder.decoding_graph.graph, atypical_nodes=code.string2nodes(count_key))
    
    if VERBOSE:
        print("\nmeas_str:", count_key)
        print("flipped_qubit_dict", flipped_qubit_dict)
        print("logical:", logical, "\n")
        if logical[0] == 1:
            print("------------ Wrong logical! ------------\n")

    

print( "\nLogical_counts:", logical_counts)


  0%|          | 4/10000 [00:00<11:48, 14.11it/s]

{'0': 1, '1': 1}


  1%|          | 112/10000 [00:08<11:38, 14.16it/s]

{'0': 107, '1': 2}


  1%|          | 124/10000 [00:08<12:23, 13.28it/s]

...

{'0': 1963, '1': 28}


 20%|██        | 2044/10000 [02:25<11:17, 11.74it/s]

{'0': 2012, '1': 29}


 20%|██        | 2050/10000 [02:25<09:24, 14.07it/s]


KeyboardInterrupt: 

# PM

In [11]:
import numpy as np
import stim
import pymatching

from soft_info import get_repcode_layout, get_KDEs, draw_matching_graph

# Code parameters
d=max_distance
T=max_distance
layout = get_repcode_layout(distance=max_distance, backend=provider.get_backend("ibm_sherbrooke"), _is_hex=True)

kde_dict, scaler_dict = get_KDEs(provider, 'ibm_sherbrooke', layout, bandwidths=0.2, plot=False)

circuit = stim.Circuit.generated("repetition_code:memory",
                                 distance=d,
                                 rounds=T,
                                 after_clifford_depolarization=0.1)

model = circuit.detector_error_model(decompose_errors=True)
matching = pymatching.Matching.from_detector_error_model(model)

In [20]:
from tqdm import tqdm
from soft_info import get_counts, soft_reweight_pymatching, counts_to_det_syndr, draw_matching_graph, reweight_edges_to_one

VERBOSE = False

num_errors = 0

## For Testing purposes ## 
#reweight_edges_to_one(matching)
IQ_data = memory[1]
soft_reweight_pymatching(matching, d, T, IQ_data, kde_dict, layout, scaler_dict, common_measure=0.01, verbose=False) 

i = 0
w_idx_lst = []
for shot in tqdm(range(len(memory))[:]):
    i += 1
    IQ_data = memory[shot]

    #counts = get_counts([IQ_data], kde_dict, scaler_dict, layout, T, verbose=False)
    #count_key = next(iter(counts.keys()))
    
    #actual_observables = [(int(count_key[0])+0)%2]
    soft_reweight_pymatching(matching, d, T, IQ_data, kde_dict, layout, scaler_dict, common_measure=0.01, verbose=False)  
    #reweight_edges_to_one(matching)

    #array_processed_string = counts_to_det_syndr(count_key, _resets=False, verbose=False)

    #predicted_observables = matching.decode(array_processed_string)

    #if predicted_observables == actual_observables: #== [0]:
        #continue
    #num_errors += 1
    #print("actual_observables:", actual_observables)
    #print("predicted_observables:", predicted_observables)
    #print("num_errors:", num_errors)

    #print(f"Wrong decoding at index {i}")
    w_idx_lst.append(i)

    
    if VERBOSE:
        print("Count key:", count_key)
        print("count_string_to_syndromes:", array_processed_string)
        print("actual_observables:", actual_observables)
        print("predicted_observables:", predicted_observables)

    
    if VERBOSE:
        matched_edges = matching.decode_to_edges_array(array_processed_string)
        print("matched_edges: ", matched_edges)
        print("Estimated flip:", predicted_observables)
        
    if VERBOSE:
        draw_matching_graph(matching, d, T, syndromes=array_processed_string, matched_edges=matched_edges, figsize=(10, 10))


print("Num errors:", num_errors)

  0%|          | 32/10000 [00:04<21:00,  7.90it/s]


KeyboardInterrupt: 